In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)
root_dir = "/content/drive/MyDrive/"
base_dir = root_dir + "tesi_magistrale/datasets/to_zip/"
path_corpus_data = root_dir + "tesi_magistrale/datasets/corpus_data_folder/"
figer_path = base_dir + "FIGER/"

Mounted at /content/drive/


In [ ]:
import json

with open(figer_path + "train.json", 'r') as inp:
    examples_train = [json.loads(l) for l in inp.readlines()]

In [ ]:
print("il dataset train contiene ",len(examples_train), "elementi")

il dataset train contiene  1505765 elementi


In [ ]:
entire_dataset_list = examples_train
print(" uniti train e test contengono ", len(entire_dataset_list), "elementi")

 uniti train e test contengono  1506199 elementi


In [ ]:
import itertools
phrase = []
phrases = []
permutations = []
num_total_phrases = 0
for i in range(0, 4):
  all_list = []
  if len(entire_dataset_list[i]['mentions'])>1:

    # creo tutte le permutazioni
    for j in entire_dataset_list[i]['mentions']:
      all_list.append(j['labels'])
    permutations = list(itertools.product(*all_list))

    num_total_phrases += len(permutations)

    # creo tutte le possibili frasi a partire dalle permutazioni
    for p in permutations:
      phrase = entire_dataset_list[i]['tokens'].copy()
      for h in range(0, len(p)):
        phrase[entire_dataset_list[i]['mentions'][h]['start']] = p[h].upper()
      
      phrases.append(phrase)
    
  else:
    for k in entire_dataset_list[i]['mentions'][0]['labels']:
      phrase = entire_dataset_list[i]['tokens'].copy()
      phrase[entire_dataset_list[i]['mentions'][0]['start']] = k.upper()
      phrases.append(phrase)
      num_total_phrases+=1

print("numero totale di frasi", num_total_phrases)
print("controllo se il numero di frasi totali è corretto", num_total_phrases == len(phrases))
print(phrases)

numero totale di frasi 51
controllo se il numero di frasi totali è corretto True
[['The', 'band', 'also', 'shared', 'membership', 'with', 'the', 'similar', ',', 'defunct', 'group', '/PERSON/MUSICIAN', 'Hud', '(', 'including', 'Tyler', 'Pope', ',', 'who', 'has', 'played', 'with', '/PERSON/MUSICIAN', 'Soundsystem', 'and', 'written', 'music', 'for', '/PERSON/MUSICIAN', ')', '.'], ['The', 'band', 'also', 'shared', 'membership', 'with', 'the', 'similar', ',', 'defunct', 'group', '/PERSON/MUSICIAN', 'Hud', '(', 'including', 'Tyler', 'Pope', ',', 'who', 'has', 'played', 'with', '/PERSON/MUSICIAN', 'Soundsystem', 'and', 'written', 'music', 'for', '/PERSON/ARTIST', ')', '.'], ['The', 'band', 'also', 'shared', 'membership', 'with', 'the', 'similar', ',', 'defunct', 'group', '/PERSON/MUSICIAN', 'Hud', '(', 'including', 'Tyler', 'Pope', ',', 'who', 'has', 'played', 'with', '/PERSON/MUSICIAN', 'Soundsystem', 'and', 'written', 'music', 'for', '/PERSON', ')', '.'], ['The', 'band', 'also', 'shared', '

In [ ]:
from gensim.parsing.preprocessing import remove_stopwords
with open(path_corpus_data + "corpus_data_figer.txt", "w+") as fp:
  for p in phrases:
    fp.write(' '.join(p) + '\n')